# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import MinMaxScaler

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
ruta_proyecto = 'C:/Users/alvar/Proyectos ML/03_RIESGOS'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

Recordamos lo que habíamos identificado en fases anteriores y lo organizamos por tipo de acción.

Modificar con pandas/numpy:

* Crear la target a partir de estado
* Discretizar num_meses_desde_ult_retraso
* En vivienda juntar NONE y OTHER
* En finalidad juntar renewable_energy, educational y vacation en 'otros'

Modificar con scikit learn:

* La variable descripcion podríamos trabajarla con un TF-IDF Vectorizer
* Convertir num_derogatorios a binaria

## CREACIÓN DE VARIABLES Y TRANSFORMACIONES CON PANDAS

### Target para el modelo EAD

In [5]:
num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139482 entries, 137387967 to 94394801
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ingresos                     139482 non-null  float64
 1   dti                          139482 non-null  float64
 2   num_hipotecas                139482 non-null  float64
 3   num_lineas_credito           139482 non-null  float64
 4   porc_tarjetas_75p            139482 non-null  float64
 5   porc_uso_revolving           139482 non-null  float64
 6   num_cancelaciones_12meses    139482 non-null  int32  
 7   num_derogatorios             139482 non-null  int32  
 8   num_meses_desde_ult_retraso  139482 non-null  float64
 9   principal                    139482 non-null  float64
 10  tipo_interes                 139482 non-null  float64
 11  imp_cuota                    139482 non-null  float64
 12  imp_amortizado               139482 non-null  fl

In [6]:
num['target_ead']=num.apply(lambda x: ((x['principal']-x['imp_amortizado'])/x['principal']) if ((x['principal']-x['imp_amortizado'])/x['principal'])>=0 else 0 ,axis=1)


In [7]:
num['target_ead'].describe().T


count    139482.000000
mean          0.337500
std           0.369130
min           0.000000
25%           0.000000
50%           0.130850
75%           0.725233
max           1.000000
Name: target_ead, dtype: float64

### Target para el modelo LGD

num.info()

In [8]:
num['target_lgd']=num.apply(lambda x: (1-(x['imp_recuperado']/(x['principal']-x['imp_amortizado'])))
                        if (1-(x['imp_recuperado']/(x['principal']-x['imp_amortizado'])))>=0 else 0,axis=1)

C:\Users\alvar\AppData\Local\Temp\ipykernel_19924\3665557246.py:2: RuntimeWarning: invalid value encountered in double_scalars
  if (1-(x['imp_recuperado']/(x['principal']-x['imp_amortizado'])))>=0 else 0,axis=1)


In [9]:
num['target_lgd'].isna().sum()

0

Comprobamos las targets creadas.

In [10]:
cat['target_pd'].mean()

0.11874650492536672

In [11]:
num[['target_ead','target_lgd']].describe().T

,count,mean,std,min,25%,50%,75%,max
target_ead,139482.0,0.337500,0.369130,0.0,0.0,0.130850,0.725233,1.0
target_lgd,139482.0,0.508857,0.490273,0.0,0.0,0.850842,1.000000,1.0


Eliminamos las variables que crearon la target

In [12]:
num.drop(columns=['imp_recuperado','imp_amortizado'],inplace=True)

In [13]:
num.drop(columns='num_meses_desde_ult_retraso',inplace=True)

## TRANSFORMACIÓN DE CATEGÓRICAS

In [14]:
cat.columns.to_list()

['empleo',
 'antiguedad_empleo',
 'ingresos_verificados',
 'rating',
 'vivienda',
 'descripcion',
 'finalidad',
 'num_cuotas',
 'target_pd']

### One Hot Encoding

#### Variables a aplicar OHE

In [15]:
var_ohe = [ 'ingresos_verificados', 'vivienda','finalidad','num_cuotas']

#### Instanciar

In [17]:
ohe = OneHotEncoder(sparse= False, handle_unknown='ignore')

#### Entrenar y aplicar

In [18]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [19]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Ordinal Encoding

#### Variables a aplicar OE

In [20]:
var_oe = ['antiguedad_empleo','rating']

#### Orden de los valores de las variables

In [21]:
#Orden de la primera variable
orden_antiguedad_empleo = ['desconocido','< 1 year','1 year','2 years','3 years','4 years',
                           '5 years','6 years','7 years','8 years','9 years','10+ years']

#Orden de la segunda variable
orden_rating =['G','F','E','D','C','B','A']

#### Instanciar

In [22]:
oe = OrdinalEncoder(categories = [orden_antiguedad_empleo,orden_rating],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 12)

#### Entrenar y aplicar

In [23]:
cat_oe = oe.fit_transform(cat[var_oe])

#### Guardar como dataframe

In [24]:
#Añadir sufijos a los nombres
nombres_oe = [variable + '_oe' for variable in var_oe]

#Guardar como dataframe
cat_oe = pd.DataFrame(cat_oe, columns = nombres_oe)

In [25]:
cat_oe

,antiguedad_empleo_oe,rating_oe
0,4.0,6.0
1,8.0,5.0
2,11.0,3.0
3,11.0,4.0
4,11.0,6.0
...,...,...
139477,11.0,4.0
139478,7.0,5.0
139479,11.0,3.0
139480,9.0,2.0


### Fechas y textos

Cargamos stopwords y limpiamos

In [26]:
stop = pd.read_csv(ruta_proyecto + '/01_Documentos/stop_words_english.txt',names = ['termino'])

def quitar_tildes(palabra):
    #Definimos la versión con tildes y símbolos y la sin
    con = 'áéíóúüñÁÉÍÓÚÜÑ'
    sin = 'aeiouunAEIOUUN'
    #Creamos un traductor
    traductor = str.maketrans(con,sin)
    #Aplicamos el traductor y devolvemos la palabra limpia
    return(palabra.translate(traductor))

stop['limpias'] = stop.termino.transform(quitar_tildes)

Aplicamos tf-idf.

In [27]:
#Instanciamos
tfidf = TfidfVectorizer(strip_accents = 'unicode',
                     stop_words = stop.limpias.to_list(),
                     max_df = 0.7,
                     min_df = 50,
                     ngram_range = (1,3),
                     max_features = 50)

#Entrenamos y aplicamos
descripcion = tfidf.fit_transform(cat.descripcion)

C:\Users\alvar\miniconda3\envs\riesgos\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'daren', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'itse', 'll', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


In [28]:
tfidf.vocabulary_

{'borrower': 15,
 '08': 7,
 '01': 0,
 '12': 11,
 'br': 22,
 '02': 1,
 'loan': 38,
 'br br': 24,
 'br borrower': 23,
 'br br borrower': 25,
 '11': 10,
 '14': 13,
 'credit': 31,
 'card': 26,
 'debt': 35,
 'pay': 42,
 'payment': 45,
 'cards': 28,
 'consolidate': 29,
 'payments': 46,
 'month': 40,
 'credit card': 32,
 'card debt': 27,
 'credit cards': 34,
 'credit card debt': 33,
 '07': 6,
 '13': 12,
 'years': 49,
 'borrower 07': 20,
 'pay credit': 43,
 '03': 2,
 'consolidation': 30,
 'borrower 03': 18,
 'bills': 14,
 'monthly': 41,
 'paying': 44,
 'rate': 47,
 'borrower 12': 21,
 '09': 8,
 'job': 37,
 'time': 48,
 '10': 9,
 '06': 5,
 '05': 4,
 'borrower 06': 19,
 'money': 39,
 'high': 36,
 '04': 3,
 'borrower 02': 17,
 'borrower 01': 16}

No parece que estos términos puedan generar información adicional a lo que tenemos, así que no incluiremos esta variable en los modelos.

In [29]:
#Eliminamos de cat las variables que no vamos a usar
cat=cat.drop(columns=['empleo','descripcion'])

## TRANSFORMACIÓN DE NUMÉRICAS

In [30]:
num.describe().T

,count,mean,std,min,25%,50%,75%,max
ingresos,139482.0,76268.694184,45938.700153,0.00,46000.00,65000.000000,93000.000000,432000.00
dti,139482.0,18.558326,9.292979,0.00,11.93,17.780000,24.430000,100.00
num_hipotecas,139482.0,1.513794,1.885264,0.00,0.00,1.000000,2.000000,25.00
num_lineas_credito,139482.0,11.613807,5.647946,0.00,8.00,11.000000,14.000000,84.00
porc_tarjetas_75p,139482.0,41.053049,36.431064,0.00,0.00,33.300000,66.700000,100.00
porc_uso_revolving,139482.0,50.259137,24.684028,0.00,31.50,50.200000,69.300000,100.00
num_cancelaciones_12meses,139482.0,0.007872,0.088375,0.00,0.00,0.000000,0.000000,1.00
num_derogatorios,139482.0,0.158472,0.365184,0.00,0.00,0.000000,0.000000,1.00
principal,139482.0,15016.367524,9163.323034,500.00,8000.00,12800.000000,20000.000000,40000.00
tipo_interes,139482.0,13.093880,4.829560,5.31,9.49,12.620000,15.990000,30.99


In [31]:
num.columns.to_list()

['ingresos',
 'dti',
 'num_hipotecas',
 'num_lineas_credito',
 'porc_tarjetas_75p',
 'porc_uso_revolving',
 'num_cancelaciones_12meses',
 'num_derogatorios',
 'principal',
 'tipo_interes',
 'imp_cuota',
 'target_ead',
 'target_lgd']

## REESCALAR VARIABLES

### Con Min-Max

In [32]:
num['num_derogatorios']


id_cliente
137387967    0
132883631    0
131289518    0
77861711     0
103448519    0
            ..
51876926     0
121031962    1
135641397    0
53664762     1
94394801     0
Name: num_derogatorios, Length: 139482, dtype: int32

In [33]:
#df de las variables que no sufren transformacion

df_otros = pd.DataFrame({
    'num_cancelaciones_12meses': num['num_cancelaciones_12meses'],
    'num_derogatorios': num['num_derogatorios']
}, index=num.index)


Unificar los datasets a reescalar.

In [34]:
num_escalar = num[['ingresos',
                  'dti',
                  'num_lineas_credito',
                  'porc_uso_revolving',
                  'principal',
                  'tipo_interes',
                  'imp_cuota',
                  'num_hipotecas',
                  'porc_tarjetas_75p']].reset_index(drop=True)

In [35]:
df_res = pd.concat([cat_oe,num_escalar], axis=1)

#### Variables a reescalar con Min-Max

In [36]:
var_mms = df_res.columns

#### Instanciar

In [37]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [38]:
df_mms = mms.fit_transform(df_res[var_mms])

#### Guardar como dataframe

In [39]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]

#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

## UNIFICAR DATASETS

### Crear una lista con los dataframes y variables a incluir en el tablón analítico para el modelo de PD

In [40]:
incluir_pd = [cat_ohe, df_mms, df_otros.reset_index().drop(columns=['id_cliente']), cat.target_pd.reset_index()]

### Unir todos los dataframes en el tablón analítico para el modelo de PD

In [41]:
df_pd = pd.concat(incluir_pd, axis = 1)

In [42]:
df_pd.set_index('id_cliente',inplace=True)

### Crear una lista con los dataframes y variables a incluir en el tablón analítico para el modelo de EAD

In [43]:
incluir_ead = [cat_ohe, df_mms, df_otros.reset_index().drop(columns=['id_cliente']), num.target_ead.reset_index()]

### Unir todos los dataframes en el tablón analítico para el modelo de EAD

In [44]:
df_ead = pd.concat(incluir_ead, axis = 1)

In [45]:
df_ead.set_index('id_cliente',inplace=True)

### Crear una lista con los dataframes y variables a incluir en el tablón analítico para el modelo de LGD

In [46]:
incluir_lgd = [cat_ohe, df_mms, df_otros.reset_index().drop(columns=['id_cliente']), num.target_lgd.reset_index()]

### Unir todos los dataframes en el tablón analítico para el modelo de LGD

In [47]:
df_lgd = pd.concat(incluir_lgd, axis = 1)

In [48]:
df_lgd.set_index('id_cliente',inplace=True)

## GUARDAR DATASETS TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [49]:
#Definir los nombres de los archivos
ruta_df_tablon_pd = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon_pd.pickle'
ruta_df_tablon_ead = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon_ead.pickle'
ruta_df_tablon_lgd = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon_lgd.pickle'

In [50]:
# #Guardar los archivos
df_pd.to_pickle(ruta_df_tablon_pd)
df_ead.to_pickle(ruta_df_tablon_ead)
df_lgd.to_pickle(ruta_df_tablon_lgd)

In [51]:
df_lgd.shape


(139482, 33)